In [ ]:
import numpy as np
import xarray as xr
from modules.ADI import ADI
from modules.classes import Quantity2D, Analytic, Interpolate
from modules.tests import plot_mass_conservation, calculate_boundary_flux, integrate_concentration
from modules.functions import animate
import matplotlib.pyplot as plt
from modules.tests import test_gaussian
import xarray as xr

In [ ]:
# Initialize a constant diffusion coefficient function
'''
diffusion = Analytic(lambda x, y: y**2)
diffusion.set_partial_x(lambda x, y: y)
diffusion.set_partial_y(lambda x, y: x)
'''


In [ ]:
n_grid = 100
n_time = 500
dt = 0.0001
dx = 0.005
dy = 0.005
conc = Quantity2D(
    n_grid,
    n_time,
    (-0.5*n_grid*dx, 0.5*n_grid*dx),
    (-0.5*n_grid*dy, 0.5*n_grid*dy),
    (0, dt*n_time),
)

diffarr = 0.02*np.ones((10, 10))
diffarr[:,4:-4] = 0.2 # Vary along x; partial_x should be constant
xint = np.linspace(-0.5*n_grid*dx, 0.5*n_grid*dx, 10)
yint = np.linspace(-0.5*n_grid*dy, 0.5*n_grid*dy, 10)
diffusion = Interpolate(diffarr, xint, yint, s=0)

diffusion.plot_2D(func='func')

In [ ]:
xcoords = conc.xcoords
ycoords = conc.ycoords
X, Y = np.meshgrid(xcoords, ycoords)
initial_condition = 1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)
sources =  1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)
#initial_condition = np.empty_like(X)
#initial_condition[n_grid//2, n_grid//2] = 100

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, sources)

In [ ]:
result_ds = ADI(conc, diffusion, initial_condition, sources, BC='dirichlet')
# Issue with explicit component vectors blowing up to infinity/nan values

In [ ]:
result_ds

In [ ]:
result_ds.to_netcdf('test_file.nc')

In [ ]:
xr.plot.contourf(result_ds['concentration'].isel(t=100), levels=30)

In [ ]:
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

def animate(ds, vmin=None, vmax=None, cmap='RdBu_r', levels=30):
    if not vmax:
        vmax = ds['concentration'].max()
        vmin = -vmax

    n_time = ds.attrs['n_time']
    fig, ax = plt.subplots()
    div = make_axes_locatable(ax)
    cax = div.append_axes('right', '5%', '5%')
    x = ds.coords['x']
    y = ds.coords['y']
    tcoords = ds.coords['t'].values
    z = ds['concentration'].values
    conc = ax.contourf(x, y, z[:,:,0], cmap=cmap, levels=levels)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    def animate(t):
        ax.clear()
        #cax.cla()
        ax.set_title("time = %.3g" % tcoords[t])
        conc = ax.contourf(x, y, z[:,:,t], cmap=cmap, levels=levels)
        #cbar = fig.colorbar(conc)
        #conc.set_array(z[:,:,t])
        return conc,
    anim = animation.FuncAnimation(fig, animate, frames = n_time, interval=1, repeat=True, blit=False)
    writergif = animation.PillowWriter(fps=20)
    return anim, writergif

In [ ]:
'''
anim, writergif = animate(result_ds, levels=100)
anim.save(r'test.gif',writer=writergif)
plt.show()
'''

In [ ]:
fig, ax = plot_mass_conservation(result_ds)

In [ ]:
test_int, test_analytic = test_gaussian(ADI, BC='dirichlet')
diff = test_int - test_analytic

In [ ]:
xr.plot.contourf(diff.isel(t=100), levels=30)

In [ ]:
absdiff = np.absolute(diff)
absdiff.median(dim=('x','y')).plot()

In [ ]:
absdiff.mean(dim=('x','y')).plot()